In [1]:
from langchain_community.document_loaders import PyPDFLoader
from plum_chatbot.configs.folders import PDF_DIR

In [2]:
from pypdf import PdfReader

reader = PdfReader(PDF_DIR / "1. COME CONFIGURARE L'EMAIL E LA PEC.pdf")

page = reader.pages[0]

for count, image_file_object in enumerate(page.images):
    print(image_file_object)

ImageFile(name=Im13.png, data: 47.3 kB)
ImageFile(name=Im4.png, data: 53.2 kB)
ImageFile(name=Im5.png, data: 585 Byte)
ImageFile(name=Im6.png, data: 20.8 kB)


In [3]:
pages = []
files = list(filter(lambda x: 'merged' not in str(x), PDF_DIR.glob("*.pdf")))

for document in files:
    loader = PyPDFLoader(PDF_DIR / document)
    content = ""
    async for page in loader.alazy_load():
        # pages.append(page)
        content += page.page_content + "\n"
    pages.append(content)

In [4]:
from qdrant_client import QdrantClient, models
import ollama

from plum_chatbot.configs.settings import Settings

settings = Settings()

COLLECTION_NAME = "FAQ"

# Initialize Ollama client
oclient = ollama.Client(host="host.docker.internal")

# Initialize Qdrant client
qclient = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)


/tmp/ipykernel_35690/877854893.py:14: UserWarning: Api key is used with an insecure connection.
  qclient = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)


In [5]:
# Text to embed

# Generate embeddings
# page_contents = [document.page_content for document in pages]
responses = [oclient.embeddings(model="llama3.2", prompt=content) for content in pages]
embeddings = [response["embedding"] for response in responses]

# Create a collection if it doesn't already exist
if not qclient.collection_exists(COLLECTION_NAME):
    qclient.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=len(embeddings[0]), distance=models.Distance.COSINE
        ),
    )

In [6]:
# Upload the vectors to the collection along with the original text as payload

qclient.upsert(
    collection_name=COLLECTION_NAME,
    points=[models.PointStruct(id=i, vector=embedding, payload={"page_content": text}) for i, (text, embedding) in enumerate(zip(pages, embeddings)) if embedding],
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)